# Chapter 37: Building a DAO Governance System

---

Decentralized Autonomous Organizations (DAOs) represent a paradigm shift in how communities coordinate and make decisions. A DAO is an organization governed by smart contracts, where token holders propose and vote on actions, and code executes the approved decisions. In this chapter, we'll build a complete DAO governance system from scratch using OpenZeppelin's battle-tested Governor contracts. Our DAO will include a governance token, a governor contract with voting and proposal mechanisms, a timelock for security, and a treasury to hold assets. We'll also develop a simple frontend to interact with the DAO. By the end, you'll have a functional DAO and a deep understanding of how on-chain governance works.

---

## 37.1 Project Overview

### 37.1.1 Requirements and Features

Our DAO will include the following features:

- **Governance Token**: An ERC-20 token with voting capabilities (ERC20Votes) that allows token holders to delegate their voting power and vote on proposals.
- **Proposal Creation**: Any token holder with sufficient voting power (proposal threshold) can create a proposal describing a set of actions (e.g., transferring funds, updating parameters, upgrading contracts).
- **Voting**: Token holders can vote "For", "Against", or "Abstain" on proposals during a defined voting period.
- **Quorum**: A minimum percentage of total supply must participate for a vote to be valid.
- **Timelock**: Approved proposals are subject to a timelock delay before execution, giving users time to exit if they disagree.
- **Treasury**: A contract holding assets (ETH and ERC-20 tokens) controlled by the DAO via the timelock.
- **Execution**: After the timelock delay, anyone can execute the proposal, which triggers the encoded actions.

### 37.1.2 Architecture Design

We'll use OpenZeppelin's governance contracts, which are modular and composable.

```
┌─────────────────────────────────────────────────────────────┐
│                      Governance Token                        │
│  • ERC20Votes                                                │
│  • Delegation                                                │
│  • Vote tracking                                             │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                         Governor                             │
│  • Proposal creation                                        │
│  • Voting                                                   │
│  • Queueing (to timelock)                                   │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                      Timelock Controller                     │
│  • Delays execution                                         │
│  • Holds treasury assets                                    │
│  • Executes after delay                                     │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                         Treasury                             │
│  • (May be the timelock itself or a separate contract)      │
└─────────────────────────────────────────────────────────────┘
```

**Components:**

- **GovernanceToken**: An ERC-20 token that inherits from OpenZeppelin's `ERC20Votes` and `ERC20Permit` (for gasless approvals).
- **Governor**: The core governance contract, combining several extensions:
  - `GovernorSettings`: Sets voting delay, voting period, and proposal threshold.
  - `GovernorCountingSimple`: Simple for/against/abstain vote counting.
  - `GovernorVotes`: Integrates with the voting token.
  - `GovernorVotesQuorumFraction`: Sets quorum as a percentage of total supply.
  - `GovernorTimelockControl`: Connects to a timelock controller.
- **TimelockController**: Holds funds and enforces a delay between proposal approval and execution.
- **Treasury**: The timelock itself acts as the treasury; any contract that needs to receive funds from the DAO should send them to the timelock address.

---

## 37.2 Smart Contract Development

We'll build the contracts using OpenZeppelin's upgradeable contracts (though for simplicity, we'll use non-upgradeable here; in production you might want upgradeability).

### 37.2.1 Governance Token

The governance token must support voting and delegation. OpenZeppelin's `ERC20Votes` extension provides this.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Permit.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Votes.sol";

contract GovernanceToken is ERC20, ERC20Permit, ERC20Votes {
    constructor(uint256 initialSupply) ERC20("GovernanceToken", "GOV") ERC20Permit("GovernanceToken") {
        _mint(msg.sender, initialSupply);
    }

    // The following functions are overrides required by Solidity.
    function _afterTokenTransfer(address from, address to, uint256 amount)
        internal
        override(ERC20, ERC20Votes)
    {
        super._afterTokenTransfer(from, to, amount);
    }

    function _mint(address to, uint256 amount)
        internal
        override(ERC20, ERC20Votes)
    {
        super._mint(to, amount);
    }

    function _burn(address account, uint256 amount)
        internal
        override(ERC20, ERC20Votes)
    {
        super._burn(account, amount);
    }
}
```

**Explanation:**
- `ERC20Permit` allows gasless approvals (EIP-2612), useful for meta-transactions.
- `ERC20Votes` tracks voting power at each block, enabling delegation and vote snapshots.
- The constructor mints the initial supply to the deployer.

### 37.2.2 Governor Contract

The governor is the heart of the DAO. We'll create a custom governor by inheriting from OpenZeppelin's `Governor` and adding the necessary extensions.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/governance/Governor.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorSettings.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorCountingSimple.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorVotes.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorVotesQuorumFraction.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorTimelockControl.sol";

contract MyGovernor is
    Governor,
    GovernorSettings,
    GovernorCountingSimple,
    GovernorVotes,
    GovernorVotesQuorumFraction,
    GovernorTimelockControl
{
    constructor(IVotes _token, TimelockController _timelock)
        Governor("MyGovernor")
        GovernorSettings(1 /* 1 block voting delay */, 50400 /* 1 week ~ 50400 blocks */, 0 /* proposal threshold */)
        GovernorVotes(_token)
        GovernorVotesQuorumFraction(4) // 4% quorum
        GovernorTimelockControl(_timelock)
    {}

    // The following functions are overrides required by Solidity.
    function votingDelay()
        public
        view
        override(IGovernor, GovernorSettings)
        returns (uint256)
    {
        return super.votingDelay();
    }

    function votingPeriod()
        public
        view
        override(IGovernor, GovernorSettings)
        returns (uint256)
    {
        return super.votingPeriod();
    }

    function quorum(uint256 blockNumber)
        public
        view
        override(IGovernor, GovernorVotesQuorumFraction)
        returns (uint256)
    {
        return super.quorum(blockNumber);
    }

    function proposalThreshold()
        public
        view
        override(Governor, GovernorSettings)
        returns (uint256)
    {
        return super.proposalThreshold();
    }

    function state(uint256 proposalId)
        public
        view
        override(Governor, GovernorTimelockControl)
        returns (ProposalState)
    {
        return super.state(proposalId);
    }

    function _execute(uint256 proposalId, address[] memory targets, uint256[] memory values, bytes[] memory calldatas, bytes32 descriptionHash)
        internal
        override(Governor, GovernorTimelockControl)
    {
        super._execute(proposalId, targets, values, calldatas, descriptionHash);
    }

    function _cancel(address[] memory targets, uint256[] memory values, bytes[] memory calldatas, bytes32 descriptionHash)
        internal
        override(Governor, GovernorTimelockControl)
        returns (uint256)
    {
        return super._cancel(targets, values, calldatas, descriptionHash);
    }

    function _executor()
        internal
        view
        override(Governor, GovernorTimelockControl)
        returns (address)
    {
        return super._executor();
    }
}
```

**Explanation:**
- `GovernorSettings` sets the voting delay (time between proposal creation and start of voting), voting period, and proposal threshold (minimum votes needed to create a proposal).
- `GovernorCountingSimple` provides simple for/against/abstain vote counting.
- `GovernorVotes` integrates with the voting token (our `GovernanceToken`).
- `GovernorVotesQuorumFraction` sets quorum as a percentage of total supply (4% in this example).
- `GovernorTimelockControl` connects to a timelock; proposals must go through the timelock before execution.

### 37.2.3 Timelock Controller

The timelock adds a safety delay between proposal approval and execution. It also holds the treasury assets.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/governance/TimelockController.sol";

contract MyTimelock is TimelockController {
    constructor(
        uint256 minDelay,
        address[] memory proposers,
        address[] memory executors,
        address admin
    ) TimelockController(minDelay, proposers, executors, admin) {}
}
```

**Parameters:**
- `minDelay`: Minimum delay in seconds before execution (e.g., 2 days = 172800 seconds).
- `proposers`: Addresses allowed to schedule proposals (should include the governor).
- `executors`: Addresses allowed to execute after delay (can be anyone, or restrict).
- `admin`: Address that can change the timelock's settings (usually the deployer initially, later transferred to a multi-sig or the governor itself).

### 37.2.4 Treasury Contract

In our setup, the timelock itself acts as the treasury. It holds all assets (ETH and tokens) owned by the DAO. Any contract that needs to receive funds from the DAO should be designed to receive from the timelock.

If you need a separate treasury with additional logic (e.g., streaming payments), you can create a custom contract controlled by the timelock. For simplicity, we'll use the timelock as the treasury.

---

## 37.3 Deployment and Setup

### 37.3.1 Deployment Script

We'll deploy in this order:
1. Deploy `GovernanceToken` (mint initial supply to deployer).
2. Deploy `MyTimelock` with proposers = [governor address (to be set later)], executors = [address(0)] (anyone), admin = deployer (or multi-sig).
3. Deploy `MyGovernor` with token and timelock addresses.
4. Grant roles in timelock: `PROPOSER_ROLE` to governor, `EXECUTOR_ROLE` to address(0) (anyone) or to governor.
5. Transfer token ownership (if any) to timelock or multi-sig as needed.
6. Transfer timelock admin role to a multi-sig or DAO itself.

**Hardhat deployment script:**
```javascript
const { ethers } = require("hardhat");

async function main() {
  const [deployer] = await ethers.getSigners();
  console.log("Deploying with account:", deployer.address);

  // Deploy GovernanceToken
  const initialSupply = ethers.parseEther("1000000"); // 1 million tokens
  const Token = await ethers.getContractFactory("GovernanceToken");
  const token = await Token.deploy(initialSupply);
  await token.waitForDeployment();
  console.log("GovernanceToken deployed to:", await token.getAddress());

  // Deploy Timelock with minDelay 2 days (172800 seconds)
  const minDelay = 2 * 24 * 60 * 60; // 2 days
  const proposers = []; // will add governor later
  const executors = [ethers.ZeroAddress]; // anyone can execute
  const admin = deployer.address; // initially deployer, will transfer later
  const Timelock = await ethers.getContractFactory("MyTimelock");
  const timelock = await Timelock.deploy(minDelay, proposers, executors, admin);
  await timelock.waitForDeployment();
  console.log("Timelock deployed to:", await timelock.getAddress());

  // Deploy Governor
  const Governor = await ethers.getContractFactory("MyGovernor");
  const governor = await Governor.deploy(await token.getAddress(), await timelock.getAddress());
  await governor.waitForDeployment();
  console.log("Governor deployed to:", await governor.getAddress());

  // Grant proposer role to governor in timelock
  const PROPOSER_ROLE = await timelock.PROPOSER_ROLE();
  await timelock.grantRole(PROPOSER_ROLE, await governor.getAddress());
  console.log("Granted PROPOSER_ROLE to governor");

  // Grant executor role to anyone (or restrict to governor)
  const EXECUTOR_ROLE = await timelock.EXECUTOR_ROLE();
  await timelock.grantRole(EXECUTOR_ROLE, ethers.ZeroAddress);
  console.log("Granted EXECUTOR_ROLE to anyone");

  // Optionally, revoke admin role from deployer after setting up multi-sig
  // For now, keep it for simplicity

  console.log("Deployment complete!");
}

main().catch(console.error);
```

### 37.3.2 Testing the DAO

We should write tests to ensure proposals can be created, voted on, and executed.

**Hardhat test example:**
```javascript
const { expect } = require("chai");
const { ethers } = require("hardhat");

describe("DAO", function () {
  let token, timelock, governor;
  let owner, addr1, addr2;

  beforeEach(async function () {
    [owner, addr1, addr2] = await ethers.getSigners();

    // Deploy token
    const Token = await ethers.getContractFactory("GovernanceToken");
    token = await Token.deploy(ethers.parseEther("1000000"));
    // Mint some to addr1 and addr2 for testing
    await token.transfer(addr1.address, ethers.parseEther("10000"));
    await token.transfer(addr2.address, ethers.parseEther("10000"));

    // Deploy timelock
    const minDelay = 2 * 24 * 60 * 60; // 2 days
    const Timelock = await ethers.getContractFactory("MyTimelock");
    timelock = await Timelock.deploy(minDelay, [], [ethers.ZeroAddress], owner.address);

    // Deploy governor
    const Governor = await ethers.getContractFactory("MyGovernor");
    governor = await Governor.deploy(await token.getAddress(), await timelock.getAddress());

    // Grant roles
    const PROPOSER_ROLE = await timelock.PROPOSER_ROLE();
    await timelock.grantRole(PROPOSER_ROLE, await governor.getAddress());
    const EXECUTOR_ROLE = await timelock.EXECUTOR_ROLE();
    await timelock.grantRole(EXECUTOR_ROLE, ethers.ZeroAddress);

    // Delegate voting power (users must delegate to themselves or others)
    await token.connect(addr1).delegate(addr1.address);
    await token.connect(addr2).delegate(addr2.address);
    // Mine some blocks to update voting power
    await ethers.provider.send("evm_mine");
  });

  it("Should create a proposal", async function () {
    // Encode proposal data: transfer 1 ETH from timelock to addr1
    const targets = [await timelock.getAddress()];
    const values = [ethers.parseEther("1")];
    const calldatas = []; // no calldata, just transfer ETH
    const description = "Proposal #1: Send 1 ETH to addr1";

    const tx = await governor.connect(addr1).propose(targets, values, calldatas, description);
    const receipt = await tx.wait();

    // Find proposalId from event
    const event = receipt.logs.find(log => log.fragment && log.fragment.name === "ProposalCreated");
    const proposalId = event.args[0];

    expect(await governor.state(proposalId)).to.equal(0); // Pending
  });

  it("Should vote and execute a proposal", async function () {
    // Create proposal
    const targets = [addr2.address];
    const values = [ethers.parseEther("1")];
    const calldatas = [];
    const description = "Send 1 ETH to addr2";

    const tx = await governor.connect(addr1).propose(targets, values, calldatas, description);
    const receipt = await tx.wait();
    const event = receipt.logs.find(log => log.fragment && log.fragment.name === "ProposalCreated");
    const proposalId = event.args[0];

    // Wait for voting delay to pass (1 block)
    await ethers.provider.send("evm_mine");

    // Vote
    await governor.connect(addr1).castVote(proposalId, 1); // 1 = For
    await governor.connect(addr2).castVote(proposalId, 1);

    // Wait for voting period to end (50400 blocks? too long for test)
    // We'll adjust governor settings for test environment (e.g., shorter period)
    // For simplicity, we assume we have set a shorter voting period in test.

    // Actually, we need to move time forward. In production, we'd have a short test period.
    // Let's assume we deployed governor with 100 blocks voting period for test.
    // We'll just fast-forward:
    await ethers.provider.send("evm_increaseTime", [7 * 24 * 60 * 60]); // 1 week
    await ethers.provider.send("evm_mine");

    // Queue proposal
    const descriptionHash = ethers.id(description);
    await governor.connect(addr1).queue(targets, values, calldatas, descriptionHash);

    // Wait timelock delay
    await ethers.provider.send("evm_increaseTime", [2 * 24 * 60 * 60]); // 2 days
    await ethers.provider.send("evm_mine");

    // Execute
    await governor.connect(addr1).execute(targets, values, calldatas, descriptionHash);

    // Check that addr2 received 1 ETH
    const balance = await ethers.provider.getBalance(addr2.address);
    expect(balance).to.be.gt(ethers.parseEther("10000")); // assuming initial balance
  });
});
```

---

## 37.4 Frontend Development

We'll build a simple React frontend that allows users to:
- View proposals.
- Create proposals (if they have enough voting power).
- Vote on active proposals.
- Queue and execute passed proposals.

We'll use ethers.js and The Graph (optional) for indexing. For simplicity, we'll fetch proposals from the Governor contract events.

### 37.4.1 Proposal Creation Interface

```javascript
import { useState } from 'react';
import { useWeb3 } from '../context/Web3Context';
import { getGovernor, getToken } from '../utils/contracts';
import { ethers } from 'ethers';

export default function CreateProposal() {
  const { signer, account } = useWeb3();
  const [target, setTarget] = useState('');
  const [value, setValue] = useState('');
  const [calldata, setCalldata] = useState('');
  const [description, setDescription] = useState('');
  const [loading, setLoading] = useState(false);

  const handleCreate = async () => {
    if (!signer) return;
    setLoading(true);
    try {
      const governor = getGovernor(signer);
      const targets = [target];
      const values = [ethers.parseEther(value || '0')];
      const calldatas = calldata ? [calldata] : ['0x'];
      
      const tx = await governor.propose(targets, values, calldatas, description);
      await tx.wait();
      alert('Proposal created!');
    } catch (error) {
      console.error(error);
      alert('Failed to create proposal');
    } finally {
      setLoading(false);
  };

  return (
    <div>
      <h2>Create Proposal</h2>
      <input placeholder="Target Address" value={target} onChange={e => setTarget(e.target.value)} />
      <input placeholder="Value (ETH)" value={value} onChange={e => setValue(e.target.value)} />
      <input placeholder="Calldata (hex)" value={calldata} onChange={e => setCalldata(e.target.value)} />
      <textarea placeholder="Description" value={description} onChange={e => setDescription(e.target.value)} />
      <button onClick={handleCreate} disabled={loading}>Create</button>
    </div>
  );
}
```

### 37.4.2 Voting Interface

We need to fetch active proposals. We can listen to ProposalCreated events and store them, or use The Graph. For simplicity, we'll fetch from the contract using `proposalCount` and loop.

```javascript
import { useState, useEffect } from 'react';
import { useWeb3 } from '../context/Web3Context';
import { getGovernor } from '../utils/contracts';

export default function Proposals() {
  const { signer, provider } = useWeb3();
  const [proposals, setProposals] = useState([]);

  useEffect(() => {
    if (!provider) return;
    const fetchProposals = async () => {
      const governor = getGovernor(provider);
      const filter = governor.filters.ProposalCreated();
      const events = await governor.queryFilter(filter, -10000); // last 10000 blocks
      const fetched = events.map(e => ({
        id: e.args[0],
        proposer: e.args[1],
        targets: e.args[2],
        values: e.args[3],
        calldatas: e.args[4],
        startBlock: e.args[5],
        endBlock: e.args[6],
        description: e.args[7],
      }));
      setProposals(fetched);
    };
    fetchProposals();
  }, [provider]);

  const handleVote = async (proposalId, support) => {
    if (!signer) return;
    const governor = getGovernor(signer);
    const tx = await governor.castVote(proposalId, support);
    await tx.wait();
    alert('Vote cast');
  };

  return (
    <div>
      <h2>Proposals</h2>
      {proposals.map(p => (
        <div key={p.id.toString()}>
          <p>ID: {p.id.toString()}</p>
          <p>Description: {p.description}</p>
          <button onClick={() => handleVote(p.id, 1)}>For</button>
          <button onClick={() => handleVote(p.id, 0)}>Against</button>
          <button onClick={() => handleVote(p.id, 2)}>Abstain</button>
        </div>
      ))}
    </div>
  );
}
```

### 37.4.3 Execution Interface

After a proposal passes and the timelock delay passes, anyone can execute it. We need to fetch succeeded proposals and provide an execute button.

```javascript
const handleQueueAndExecute = async (proposalId) => {
  // This is simplified; you need targets, values, calldatas, descriptionHash
  // Better to store those when fetching proposals
};
```

For a complete frontend, you'd also display proposal state, voting results, and allow delegation.

---

## 37.5 Complete Walkthrough

**User flow:**

1. **Token holders delegate** their voting power (if they want to vote). They can delegate to themselves or to a trusted delegate.
2. **A user with sufficient voting power** creates a proposal by calling `propose` on the governor, specifying the actions (targets, values, calldata) and a description.
3. **After a voting delay**, the proposal enters the active state. Token holders vote using `castVote`.
4. **After the voting period ends**, if the proposal meets quorum and has more "For" votes than "Against", it succeeds.
5. **Anyone can queue** the succeeded proposal by calling `queue` with the same parameters. This sends it to the timelock.
6. **After the timelock delay** (e.g., 2 days), anyone can execute it by calling `execute`.
7. The timelock executes the actions, e.g., transferring ETH from its treasury.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 37 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  We built a complete DAO governance system:                    │
│    • GovernanceToken (ERC20Votes) for voting power             │
│    • Governor contract with configurable settings              │
│    • TimelockController for execution delay                    │
│    • Treasury (the timelock itself)                            │
│                                                                 │
│  Key concepts:                                                 │
│    • Delegation: token holders can delegate voting power       │
│    • Proposal lifecycle: Pending → Active → Succeeded →       │
│      Queued → Executed                                         │
│    • Quorum: minimum participation required                    │
│    • Timelock: protects users from sudden changes              │
│                                                                 │
│  Deployment order: token → timelock → governor → grant roles  │
│  Testing: simulate proposal creation, voting, execution        │
│  Frontend: create proposals, vote, execute                     │
│                                                                 │
│  This DAO can be extended with:                                │
│    • Upgradeable contracts                                     │
│    • Multi-sig for initial admin roles                         │
│    • Advanced voting strategies (quadratic, weighted)          │
│    • On-chain treasury with streaming payments                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 38 – Building a DeFi Yield Aggregator. We'll create a vault that automatically invests user funds in the highest-yielding strategies across lending protocols like Aave and Compound.